In [ ]:
import numpy as np
import pickle
import time
import os
import torch
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib_setup import set_size, configure_latex, savefig, generate_tex_figures, thiner_border

from utils import calculate_E, plot_Es, plot_max_values, plot_norms

configure_latex(style=['science', 'notebook'], global_save_path=os.getcwd() + "/porocilo/images")

In [ ]:
from methods import Method

L, T, Nt = 5, 5, 15000
N = 512
n, a_x, a_y = 0, 1, 1

lams = np.linspace(0, 10, 2)

saved_psis = []
Ms = []
for lam in lams:
    print(f'lam={lam}')
    M = Method(N=N, Nt=Nt, method_name='torch_ssfm')
    M.init_grid(L=L, T=T, backend='torch')
    M.init_pot(n=n, a_x=a_x, a_y=a_y, lam=lam)

    saved_psi = M.start_method(check_overflow=None, save_every=Nt // 100, im_time=True, renormalize_every=1)
    saved_psis.append(saved_psi.copy())
    
    M.X, M.Y = None, None
    M.k_X, M.k_Y = None, None
    Ms.append(M)
    torch.cuda.empty_cache()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=set_size(fraction=0.5, ratio='4:3'))
axs = thiner_border(axs)

El = []

for i, (saved_psi, M) in enumerate(zip(saved_psis, Ms)):
    
    E, Ek, Ep = calculate_E(saved_psi, M.grid.hr, V=M.pot.V.cpu().numpy())
    plt.plot(M.grid.t[::150], E, label=r'$\lambda$={:.2f}'.format(lams[i]), lw=0.7)
    
    print(lams[i], E[-1])
    El.append(E[-1])
    
axs.set_xlabel(r'$t$')
axs.set_ylabel('$E$')
# plt.legend(fontsize=7)
# savefig('Evst')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=set_size(fraction=0.5, ratio='4:3'))
ax = thiner_border(ax)

ax.plot(lams, El, lw=1)
ax.scatter(lams, El, s=4)
ax.set_xlabel('$\lambda$')
ax.set_ylabel('$E$')

ax.annotate(f'$a_x={a_x}, a_y={a_y}, L={L}$', xy=(0.55, 0.4), xycoords='axes fraction', fontsize=5)
ax.annotate(fr'$N \times N = {N} \times {N}$', xy=(0.55, 0.3), xycoords='axes fraction', fontsize=5)
ax.annotate(r'$\Delta t = {:.2e}$'.format(T/Nt), xy=(0.55, 0.2), xycoords='axes fraction', fontsize=5)
# savefig('EvsLam2')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=set_size(subplots=(1, 2), fraction=1, ratio='4:3'))
axs = [thiner_border(i) for i in axs]

L, T, Nt = 15, 20, 50000
N = 1024
n, a_x, a_y = 2, 5, 0
lam = 0.05

M = Method(N=N, Nt=Nt, method_name='torch_ssfm')
M.init_grid(L=L, T=T, backend='torch')
M.init_pot(n=n, a_x=a_x, a_y=a_y, lam=lam)

saved_psi = M.start_method(check_overflow=None, save_every=Nt // 200, im_time=False, renormalize_every=1)

E, Ek, Ep = calculate_E(saved_psi, M.grid.hr, V=M.pot.V.cpu().numpy())
plot_Es(E, Ek, Ep, norm=False, mode=2, axs=axs, lw=1, show=False)

k = Nt // 200

axs[0].set_xlabel(r'koraki $\times$ {}'.format(k))
axs[0].set_ylabel('skupna energija')
    
axs[1].set_xlabel(r'koraki $\times$ {}'.format(k))
axs[1].set_ylabel('energija')

# savefig('xy')

In [ ]:
from utils import calculate_xy

xy = calculate_xy(saved_psi, M.grid.X_np, M.grid.Y_np, M.grid.hr)
x, y = xy[0], xy[1]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=set_size(subplots=(1, 1), fraction=0.5, ratio='4:3'))
ax = thiner_border(ax)

ax.plot(x, y, lw=0.8, c='C5')
ax.set_ylabel(r'$\langle y \rangle$')
ax.set_xlabel(r'$\langle x \rangle$')
ax.annotate(f'$a_x={a_x}, a_y={a_y}, L={L}$', xy=(0.55, 0.28), xycoords='axes fraction', fontsize=5)
ax.annotate(fr'$N \times N = {N} \times {N}$', xy=(0.55, 0.20), xycoords='axes fraction', fontsize=5)
ax.annotate(r'$\Delta t = {:.2e}$'.format(T/Nt), xy=(0.55, 0.14), xycoords='axes fraction', fontsize=5)
ax.annotate(r'$\lambda = {}$'.format(lam), xy=(0.55, 0.06), xycoords='axes fraction', fontsize=5)
# savefig('xvsy')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=set_size(subplots=(1, 1), fraction=0.5, ratio='4:3'))
ax = thiner_border(ax)

ax.plot(M.grid.t[::k], y, lw=1, c='C0')
ax.plot(M.grid.t[::k], x, lw=1, c='C1')
ax.legend([r'$\langle y \rangle$', r'$\langle x \rangle$'])
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$\langle$koordinate$\rangle$')
# savefig('xyvst')

In [ ]:
# px = np.gradient(x, M.grid.ht)
# py = np.gradient(y, M.grid.ht)
# plt.plot(px, py)

In [ ]:
# plt.plot(range(len(px)), px / np.max(np.abs(px)))
# plt.plot(range(len(py)), py / np.max(np.abs(py)))

In [ ]:
from utils import probability_density

psis = probability_density(saved_psi, N=M.grid.X_np.shape[0])

In [ ]:
fig, axs = plt.subplots(10, 10, figsize=(30, 30))
axs = axs.flatten()

for i, psi in tqdm(enumerate(psis[::2])):
    axs[i].contourf(M.grid.X_np, M.grid.Y_np, psi, 50, cmap="viridis")
    axs[i].set_xticklabels([])
    axs[i].set_xticks([])
    axs[i].set_yticklabels([])
    axs[i].set_yticks([])
    
plt.tight_layout()
plt.savefig('lam05.png', dpi=200)